# Generating _views_ on model cards

In [1]:
import typing
from datetime import datetime
import PIL

import weave
from weave import panels

In [2]:
import wandb
import wandb.apis.reports as wb
wandb.login()

wandb: Currently logged in as: dpaiton. Use `wandb login --relogin` to force relogin


True

# Inferring defaults
 * We can infer some information to fill in default values on model cards, other fields are left blank.
 * Users can edit all fields as they please.
 * All edits are versioned for accountability.

In [3]:
path = "./fashion_mnist_results.png"
image = PIL.Image.open(path)

def get_model_card_args(entity, project, run_id, artifact_index):
    api = wandb.Api()
    run = api.run(f'{entity}/{project}/{run_id}')
    collection_name = 'fashion_style'
    run_name = run.name
    artifact = run.logged_artifacts()[0]
    updated = artifact.updated_at.strip()
    updated_dt = datetime.strptime(updated, '%Y-%m-%dT%H:%M:%S')
    path = "./fashion_mnist_results.png"
    image = PIL.Image.open(path)
    card_args = {
        'model_name': artifact.name,
        'created_by': User(name=run.entity),
        'updated_at': f'{updated_dt:%d/%m/%Y at %H:%M:%S}',
        'model_type': 'CNN',
        'application': 'Classifying clothing images by type',
        'primary_metric': TargetMetric(name='acc', direction='up'),
        'example': weave.save(image),
        'limitations': 'Not useful for realistic examples.'
    }
    full_args = {
        'entity': entity,
        'project': project,
        'run_id': run_id,
        'artifact_index': artifact_index,
        'entity_name': run.entity,
        'pil_image': image
    }
    full_args.update(card_args)
    return card_args, full_args

# Weave provides a flexible framework

#### Model cards have a lot of important information! But the amount of importance per field depends on who is reading it.

#### We want to preserve the integrety of the card, but also prioritize information based on the use case. This is accomplished by having alternative ways to *view* the card.

#### First we define the card itself, which is a weave op!

In [4]:
# This Type should be built-in to Weave, declared in weave.types
@weave.type()
class User:
    name: str

# This Type will be a built-in to Weave, declared in weave.types
@weave.type()
class TargetMetric:
    name: str
    direction: str # typing.Union['up','down']  # (TODO: enum)

class MarkdownString(weave.types.Type):
    pass

@weave.type()
class ExampleImage():
    instance_class = PIL.PngImagePlugin.PngImageFile
    instance_classes = PIL.PngImagePlugin.PngImageFile
    pass
        
@weave.type()
class ModelCard:
    model_name: str
    created_by: User
    updated_at: str  # TODO: timestamp
    model_type: str  # TODO: enum
    primary_metric: TargetMetric
    application: str
    
    # TODO: This is not general enough. It should depend on the type of the model
    example: ExampleImage
    limitations: str
        
# This should be an op, so we can call it from the UI, but I need to fix something
# to make that work
#@weave.op()
def model_card_panel(model_card: ModelCard) -> panels.Card:
    return panels.Card(
        title=model_card.model_name,
        subtitle=model_card.created_by.name,
        content=[
            panels.CardTab(
                name='Overview',
                content=panels.Group(
                    items=[
                        panels.Group(
                            prefer_horizontal=True,
                            items=[
                                panels.LabeledItem(item=model_card.updated_at, label='Last updated'),
                                panels.LabeledItem(item=model_card.model_type, label='Model type'),
                                panels.LabeledItem(item=model_card.primary_metric.name, label='Metric'),
                            ]
                        ),
                        panels.LabeledItem(item=model_card.application, label='Application'),
                        panels.LabeledItem(item=model_card.example, label='Example'),
                    ]
                )
            ),
            panels.CardTab(
                name='Limitations & Use',
                content=panels.LabeledItem(item=model_card.limitations, label='Limitations')
            ),

        ]
    )

#def generate_model_card_report(card_args)
#report = api.create_report(project='report-editing')
#report.title = 'A fabulous title'
#report.description = 'A descriptive description'

# Customers can modify the card specification from our published op!

### Once they've defined their card they can call a function to render it as a weave panel

In [5]:
# Programatically generate a model card (will additionally set defaults for user to replace)
entity = 'stacey'
project = 'digio'
run_id = '2pw9wdv6'
artifact_index = 0
card_args, full_args = get_model_card_args(entity, project, run_id, artifact_index)
card_args

{'model_name': 'baseline:v3',
 'created_by': User(name='stacey'),
 'updated_at': '08/06/2022 at 22:34:33',
 'model_type': 'CNN',
 'application': 'Classifying clothing images by type',
 'primary_metric': TargetMetric(name='acc', direction='up'),
 'example': <weave.weave_internal.ConstNodePILImageType at 0x126c5d4f0>,
 'limitations': 'Not useful for realistic examples.'}

In [6]:
# Define the model card
model_card = ModelCard(**card_args)

# Render it using the model_card_panel!
model_card_panel(model_card)

# Long-form view

### The model card specification has all of the information needed for each user type. You can find all of the information by looking at the programatically generated report!

In [7]:
api = wandb.Api()
wandb.require('report-editing:v0')

wandb: WARNING This is an experimental feature -- use with caution!


In [8]:
def generate_report(kwargs):
    report = api.create_report(project='model_cards')
    report.title = kwargs['model_name']
    report.description = kwargs['application']

    section1 = [
        wb.TableOfContents(),
        wb.H1('Model owner'),
        wb.P(kwargs['entity']),
        wb.H1('Overview'),
        wb.H2('Last Update'),
        wb.P(kwargs['updated_at']),
        wb.H2('Application'),
        wb.P(kwargs['application']),
        wb.H2('Example'),
        wb.Image(url='https://i.ibb.co/YWk9Ccj/fashion-mnist-results.png', caption='fashion-mnist-results'),
    ]

    report.blocks = section1
    report.save()
    return report

report = generate_report(full_args)

In [9]:
report.url

'https://wandb.ai/dpaiton/model_cards/reports/baseline%3Av3--VmlldzoyMTQ1MDI3'

In [10]:
# Produce a "comparison card" that only shows the difference between two model cards
#TODO: card_c = difference(card_object_a, card_object_b)